This ipynb file contains my implementation of project 2

First I am going to load package and data files

In [1]:
# import package
import numpy as np
import pandas as pd
import sklearn
import scipy
import pickle
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

In [2]:
# load data
trainset = pd.read_csv(r'COMP30027_2021_Project2_datasets/recipe_train.csv')
testset = pd.read_csv(r'COMP30027_2021_Project2_datasets/recipe_test.csv')
trainse_t_np = trainset.to_numpy()
testset_np = testset.to_numpy()
truth_label = trainset["duration_label"].to_numpy()

There are some helper function

In [ ]:
# calculate the accuracy
def calc_accuracy(truth, predicts):
    truth_count = 0
    for i in range(len(predicts)):
        if truth[i] == predicts[i]:
            truth_count += 1
            
    return truth_count/len(truth)


In [ ]:
# Produce Kaggle Submission
def submit_file(predicts, filename):
    id = list(range(1,10001))
    result = pd.DataFrame({'id': id, 'duration_label': predicts})
    result.to_csv(filename, index = False)
    return

Then I use Zero-R to provide the baseline of my models' evaluation. 
Note that Zero-R is not one of the 4 models I adopted to make prediction on data

In [ ]:
# Zero-R Baseline
labels = trainset['duration_label'].to_numpy()

labels_int = labels.astype(int)

# get the most freqeunt label
dist = np.bincount(labels_int)
perc_dist = dist/trainset.shape[0]
freq_int = dist.argmax()
freq = freq_int.astype(float)

# generate prediction for train
predict_train = [freq] * trainset.shape[0]

acc_train = calc_accuracy(labels, predict_train)

print("The accuracy of 0R baseline for trainset is", acc_train)


# generate prediction for test
predict_test = [freq] * testset.shape[0]


MODEL 1 MNB

In [7]:
# tfidf preprocessing and 20% best chi2 selection for train set
import re
from sklearn.feature_extraction.text import TfidfVectorizer
# concatenate three text
text = (trainset['name'] + trainset['steps'] + trainset['ingredients']).to_numpy()
for i in range(text.shape[0]):
    text[i] = text[i].replace("[", " ")
    text[i] = text[i].replace("]", " ")
    text[i] = text[i].replace("'", " ")

    
print('preprocessing name/step/ingr into tfidfvectorizer ')
X_train_txt, X_test_txt, y_train, y_test = train_test_split(text, truth_label, train_size=0.8, test_size=0.2, random_state=666)

vectorizer = TfidfVectorizer(stop_words = 'english')
vectorizer = vectorizer.fit(text)
X_train = vectorizer.transform(X_train_txt)
X_test = vectorizer.transform(X_test_txt)


print('selecting best 20% features by chi2 ')


from sklearn.feature_selection import SelectPercentile, chi2

select = SelectPercentile(chi2, percentile=20)
X_train_s = select.fit_transform(X_train, y_train)
X_test_s = select.transform(X_test)

# print(X_train_s.shape)
# print(X_test_s.shape)


preprocessing name/step/ingr into tfidfvectorizer 
selecting best 20% features by chi2 


In [9]:
# Model 1 MultinomialNB()
from sklearn.naive_bayes import GaussianNB, MultinomialNB
mnb = MultinomialNB()
mnb.fit(X_train_s, y_train)
print("MNB score")
mnb.score(X_test_s, y_test)

MNB score


0.704375

In [11]:
# tfidf preprocessing and 20% best chi2 selection for test set

text = (trainset['name'] + trainset['steps'] + trainset['ingredients']).to_numpy()
for i in range(trainset.shape[0]):
    text[i] = text[i].replace("[", " ")
    text[i] = text[i].replace("]", " ")
    text[i] = text[i].replace("'", " ")

text_test = (testset['name'] + testset['steps'] + testset['ingredients']).to_numpy()
for i in range(text_test.shape[0]):
    text_test[i] = text_test[i].replace("[", " ")
    text_test[i] = text_test[i].replace("]", " ")
    text_test[i] = text_test[i].replace("'", " ")

all_records = np.concatenate((text, text_test), axis = 0)

print('preprocessing name/step/ingr into tfidfvectorizer ')

vectorizer = TfidfVectorizer(stop_words = 'english')
vectorizer = vectorizer.fit(all_records)
all_records = vectorizer.transform(all_records)
X_train = vectorizer.transform(text)
X_test = vectorizer.transform(text_test)


# print(X_train.shape)
# print(X_test.shape)

print('selecting best 20% features by chi2 ')
select = SelectPercentile(chi2, percentile=20)
select = select.fit(X_train, truth_label)
X_train_s = select.transform(X_train)
X_test_s = select.transform(X_test)

# print(X_train_s.shape)
# print(X_test_s.shape)


preprocessing name/step/ingr into tfidfvectorizer 
selecting best 20% features by chi2 


In [13]:
# predict test with MNB
mnb = MultinomialNB()
mnb.fit(X_train_s, truth_label)
out = mnb.predict(X_test_s)
submit_file(out, 'tfidf_mnb.csv')


MODEL 2 SVM

In [15]:
# tfidf preprocessing and 20% best chi2 selection for train set
import re
from sklearn.feature_extraction.text import TfidfVectorizer
# concatenate three text
text = (trainset['name'] + trainset['steps'] + trainset['ingredients']).to_numpy()
for i in range(text.shape[0]):
    text[i] = text[i].replace("[", " ")
    text[i] = text[i].replace("]", " ")
    text[i] = text[i].replace("'", " ")

    
print('preprocessing name/step/ingr into tfidfvectorizer ')
X_train_txt, X_test_txt, y_train, y_test = train_test_split(text, truth_label, train_size=0.8, test_size=0.2, random_state=666)

vectorizer = TfidfVectorizer(stop_words = 'english')
vectorizer = vectorizer.fit(text)
X_train = vectorizer.transform(X_train_txt)
X_test = vectorizer.transform(X_test_txt)

# vectorizer = TfidfVectorizer(stop_words = 'english')
# X_train = vectorizer.fit_transform(X_train_txt)
# X_test = vectorizer.transform(X_test_txt)

print('selecting best 20% features by chi2 ')


from sklearn.feature_selection import SelectPercentile, chi2

select = SelectPercentile(chi2, percentile=20)
X_train_s = select.fit_transform(X_train, y_train)
X_test_s = select.transform(X_test)

# print(X_train_s.shape)
# print(X_test_s.shape)
# print(y_train.shape)
# print(y_test.shape)

preprocessing name/step/ingr into tfidfvectorizer 
selecting best 20% features by chi2 


In [16]:
from sklearn import svm
# find optimal hyperparameter
# find C
# explain multi_class
for i in range(1,11):
    linear_svm = svm.LinearSVC(multi_class = 'ovr', C=i, max_iter=1000);
    linear_svm.fit(X_train_s, y_train)
    acc = linear_svm.score(X_test_s, y_test)
    print("acc with C = ",i, "is", acc)
    
    




acc with C =  1 is 0.78425
acc with C =  2 is 0.78225
acc with C =  3 is 0.78125
acc with C =  4 is 0.77975
acc with C =  5 is 0.778375
acc with C =  6 is 0.77725
acc with C =  7 is 0.776375
acc with C =  8 is 0.775875
acc with C =  9 is 0.775
acc with C =  10 is 0.77425


In [19]:
from sklearn import svm
for i in range(100,5000,500):
    linear_svm = svm.LinearSVC(multi_class = 'ovr', C=1, max_iter=i);
    linear_svm.fit(X_train_s, y_train)
    acc = linear_svm.score(X_test_s, y_test)
    print("acc with max_iter = ",i, "is", acc)

for i in range(10,100,10):
    if (i != 10):
        linear_svm = svm.LinearSVC(multi_class = 'ovr', C=1, max_iter=i);
        linear_svm.fit(X_train_s, y_train)
        acc = linear_svm.score(X_test_s, y_test)
        print("acc with max_iter = ",i, "is", acc)
    else:
        print("fail to converge when i = 10")
    

acc with max_iter =  100 is 0.78425
acc with max_iter =  600 is 0.78425
acc with max_iter =  1100 is 0.78425
acc with max_iter =  1600 is 0.78425
acc with max_iter =  2100 is 0.78425
acc with max_iter =  2600 is 0.78425
acc with max_iter =  3100 is 0.78425
acc with max_iter =  3600 is 0.78425
acc with max_iter =  4100 is 0.78425
acc with max_iter =  4600 is 0.78425
fail to converge when i = 10
acc with max_iter =  20 is 0.784625
acc with max_iter =  30 is 0.78425
acc with max_iter =  40 is 0.784375
acc with max_iter =  50 is 0.78425
acc with max_iter =  60 is 0.78425
acc with max_iter =  70 is 0.78425
acc with max_iter =  80 is 0.78425
acc with max_iter =  90 is 0.78425


In [10]:
# Model 2 SVM model
from sklearn import svm
# best hyperparameter
linear_svm = svm.LinearSVC(multi_class = 'ovr', C=1, max_iter=1000);
linear_svm.fit(X_train_s, y_train)
acc = linear_svm.score(X_test_s, y_test)

print("Linear_svm", acc)




ValueError: Found input variables with inconsistent numbers of samples: [40000, 32000]

In [21]:
# do not run them!!! Computational cost too high
# svmrbf = svm.SVC(kernel='rbf', gamma=0.05, C=1)
# svmrbf.fit(X_train_s, y_train)
# acc = svmrbf.score(X_test_s, y_test)
# print("svm with rbf kernel", acc)

# svmp2 = svm.SVC(kernel='poly', degree=2, gamma='auto', C=1)
# svmp2.fit(X_train_s, y_train)
# acc = svmp2.score(X_test_s, y_test)
# print("svm with rbf kernel", acc)

In [8]:
# tfidf preprocessing and 20% best chi2 selection for test set

text = (trainset['name'] + trainset['steps'] + trainset['ingredients']).to_numpy()
for i in range(trainset.shape[0]):
    text[i] = text[i].replace("[", " ")
    text[i] = text[i].replace("]", " ")
    text[i] = text[i].replace("'", " ")

text_test = (testset['name'] + testset['steps'] + testset['ingredients']).to_numpy()
for i in range(text_test.shape[0]):
    text_test[i] = text_test[i].replace("[", " ")
    text_test[i] = text_test[i].replace("]", " ")
    text_test[i] = text_test[i].replace("'", " ")

all_records = np.concatenate((text, text_test), axis = 0)

print('preprocessing name/step/ingr into tfidfvectorizer ')

vectorizer = TfidfVectorizer(stop_words = 'english')
vectorizer = vectorizer.fit(all_records)
all_records = vectorizer.transform(all_records)
X_train = vectorizer.transform(text)
X_test = vectorizer.transform(text_test)


# print(X_train.shape)
# print(X_test.shape)

print('selecting best 20% features by chi2 ')
select = SelectPercentile(chi2, percentile=20)
select = select.fit(X_train, truth_label)
X_train_s = select.transform(X_train)
X_test_s = select.transform(X_test)

# print(X_train_s.shape)
# print(X_test_s.shape)


preprocessing name/step/ingr into tfidfvectorizer 
selecting best 20% features by chi2 


In [11]:

linear_svm = svm.LinearSVC(multi_class = 'ovr', C=1, max_iter=10000)
linear_svm.fit(X_train_s, truth_label)
out = linear_svm.predict(X_test_s)
submit_file(out, 'tfidf_linear_svc.csv')

MODEL 3 Logistic

In [12]:
# tfidf preprocessing and 20% best chi2 selection for train set
import re
from sklearn.feature_extraction.text import TfidfVectorizer
# concatenate three text
text = (trainset['name'] + trainset['steps'] + trainset['ingredients']).to_numpy()
for i in range(text.shape[0]):
    text[i] = text[i].replace("[", " ")
    text[i] = text[i].replace("]", " ")
    text[i] = text[i].replace("'", " ")

    
print('preprocessing name/step/ingr into tfidfvectorizer ')
X_train_txt, X_test_txt, y_train, y_test = train_test_split(text, truth_label, train_size=0.8, test_size=0.2, random_state=666)

vectorizer = TfidfVectorizer(stop_words = 'english')
vectorizer = vectorizer.fit(text)
X_train = vectorizer.transform(X_train_txt)
X_test = vectorizer.transform(X_test_txt)

# vectorizer = TfidfVectorizer(stop_words = 'english')
# X_train = vectorizer.fit_transform(X_train_txt)
# X_test = vectorizer.transform(X_test_txt)

print('selecting best 20% features by chi2 ')


from sklearn.feature_selection import SelectPercentile, chi2

select = SelectPercentile(chi2, percentile=20)
X_train_s = select.fit_transform(X_train, y_train)
X_test_s = select.transform(X_test)

print(X_train_s.shape)
print(X_test_s.shape)
print(y_train.shape)
print(y_test.shape)

preprocessing name/step/ingr into tfidfvectorizer 
selecting best 20% features by chi2 
(32000, 4809)
(8000, 4809)
(32000,)
(8000,)


In [14]:
from sklearn.linear_model import LogisticRegression
# Logistic model
i = 100
while i < 10000:
    if (i != 100):
        lgr = LogisticRegression(max_iter = i)
        lgr.fit(X_train_s, y_train)   
        print("When max_iter = ", i, "Accuracy:",lgr.score(X_test_s,y_test))
        
    else:
        print("fail to converge when i = 250")
        
    i *= 2.5

    

fail to converge when i = 250
When max_iter =  250.0 Accuracy: 0.787625
When max_iter =  625.0 Accuracy: 0.787625
When max_iter =  1562.5 Accuracy: 0.787625
When max_iter =  3906.25 Accuracy: 0.787625
When max_iter =  9765.625 Accuracy: 0.787625


In [15]:
for i in range(1,8):
    lgr = LogisticRegression(C=i, max_iter = 10000)
    lgr.fit(X_train_s, y_train)
    print("When C = ", i,"Accuracy:",lgr.score(X_test_s,y_test))



When C =  1 Accuracy: 0.787625
When C =  2 Accuracy: 0.791
When C =  3 Accuracy: 0.788625
When C =  4 Accuracy: 0.78775
When C =  5 Accuracy: 0.785875
When C =  6 Accuracy: 0.78475
When C =  7 Accuracy: 0.784625


In [16]:
lgr = LogisticRegression(C=2, max_iter = 10000)
lgr.fit(X_train_s, y_train)
print("Accuracy:",lgr.score(X_test_s,y_test))



Accuracy: 0.791


In [21]:
# tfidf preprocessing and 20% best chi2 selection for test set

text = (trainset['name'] + trainset['steps'] + trainset['ingredients']).to_numpy()
for i in range(trainset.shape[0]):
    text[i] = text[i].replace("[", " ")
    text[i] = text[i].replace("]", " ")
    text[i] = text[i].replace("'", " ")

text_test = (testset['name'] + testset['steps'] + testset['ingredients']).to_numpy()
for i in range(text_test.shape[0]):
    text_test[i] = text_test[i].replace("[", " ")
    text_test[i] = text_test[i].replace("]", " ")
    text_test[i] = text_test[i].replace("'", " ")

all_records = np.concatenate((text, text_test), axis = 0)

print('preprocessing name/step/ingr into tfidfvectorizer ')

vectorizer = TfidfVectorizer(stop_words = 'english')
vectorizer = vectorizer.fit(all_records)
all_records = vectorizer.transform(all_records)
X_train = vectorizer.transform(text)
X_test = vectorizer.transform(text_test)


# print(X_train.shape)
# print(X_test.shape)

print('selecting best 20% features by chi2 ')
select = SelectPercentile(chi2, percentile=20)
select = select.fit(X_train, truth_label)
X_train_s = select.transform(X_train)
X_test_s = select.transform(X_test)

# print(X_train_s.shape)
# print(X_test_s.shape)

preprocessing name/step/ingr into tfidfvectorizer 
selecting best 20% features by chi2 


In [22]:
# for kaggle, C=1 is optimal
lgr = LogisticRegression(C=1, max_iter = 10000);
lgr.fit(X_train_s, truth_label)
out = lgr.predict(X_test_s)
submit_file(out, 'tfidf_log_nsi.csv')
                           


In [23]:
# try name and step only
# tfidf preprocessing and 20% best chi2 selection for train set
import re
from sklearn.feature_extraction.text import TfidfVectorizer
# concatenate three text
text = (trainset['name'] + trainset['steps']).to_numpy()
for i in range(text.shape[0]):
    text[i] = text[i].replace("[", " ")
    text[i] = text[i].replace("]", " ")
    text[i] = text[i].replace("'", " ")

    
print('preprocessing name/step/ingr into tfidfvectorizer ')
X_train_txt, X_test_txt, y_train, y_test = train_test_split(text, truth_label, train_size=0.8, test_size=0.2, random_state=666)

vectorizer = TfidfVectorizer(stop_words = 'english')
vectorizer = vectorizer.fit(text)
X_train = vectorizer.transform(X_train_txt)
X_test = vectorizer.transform(X_test_txt)

# vectorizer = TfidfVectorizer(stop_words = 'english')
# X_train = vectorizer.fit_transform(X_train_txt)
# X_test = vectorizer.transform(X_test_txt)

print('selecting best 20% features by chi2 ')


from sklearn.feature_selection import SelectPercentile, chi2

select = SelectPercentile(chi2, percentile=20)
X_train_s = select.fit_transform(X_train, y_train)
X_test_s = select.transform(X_test)

print(X_train_s.shape)
print(X_test_s.shape)
print(y_train.shape)
print(y_test.shape)

preprocessing name/step/ingr into tfidfvectorizer 
selecting best 20% features by chi2 
(32000, 4766)
(8000, 4766)
(32000,)
(8000,)


In [24]:
for i in range(1,8):
    lgr = LogisticRegression(C=i, max_iter = 10000)
    lgr.fit(X_train_s, y_train)
    print("When C = ", i,"Accuracy:",lgr.score(X_test_s,y_test))


When C =  1 Accuracy: 0.788625
When C =  2 Accuracy: 0.787875
When C =  3 Accuracy: 0.787375
When C =  4 Accuracy: 0.785375
When C =  5 Accuracy: 0.78325
When C =  6 Accuracy: 0.781625
When C =  7 Accuracy: 0.78275


In [25]:
# tfidf preprocessing and 20% best chi2 selection for test set

text = (trainset['name'] + trainset['steps']).to_numpy()
for i in range(trainset.shape[0]):
    text[i] = text[i].replace("[", " ")
    text[i] = text[i].replace("]", " ")
    text[i] = text[i].replace("'", " ")

text_test = (testset['name'] + testset['steps']).to_numpy()
for i in range(text_test.shape[0]):
    text_test[i] = text_test[i].replace("[", " ")
    text_test[i] = text_test[i].replace("]", " ")
    text_test[i] = text_test[i].replace("'", " ")

all_records = np.concatenate((text, text_test), axis = 0)

print('preprocessing name/step/ingr into tfidfvectorizer ')

vectorizer = TfidfVectorizer(stop_words = 'english')
vectorizer = vectorizer.fit(all_records)
all_records = vectorizer.transform(all_records)
X_train = vectorizer.transform(text)
X_test = vectorizer.transform(text_test)


# print(X_train.shape)
# print(X_test.shape)

print('selecting best 20% features by chi2 ')
select = SelectPercentile(chi2, percentile=20)
select = select.fit(X_train, truth_label)
X_train_s = select.transform(X_train)
X_test_s = select.transform(X_test)

# print(X_train_s.shape)
# print(X_test_s.shape)

preprocessing name/step/ingr into tfidfvectorizer 
selecting best 20% features by chi2 


In [26]:

lgr = LogisticRegression(C=1, max_iter = 10000);
lgr.fit(X_train_s, truth_label)
out = lgr.predict(X_test_s)
submit_file(out, 'tfidf_log_ns.csv')
                           